# Attachment generator

In [ ]:
import os

def create_exact_size_file(file_path, size_mb):
    # Calculate the exact size in bytes
    size_bytes = size_mb * 1024 * 1024

    # Open the file in write binary ('wb') mode
    with open(file_path, 'wb') as file:
        file.seek(size_bytes - 1)  # Move to the position one byte before the desired size
        file.write(b'\0')  # Write a single zero byte at this position

def generate_files(directory_path, excel_filename, pdf_filename, target_size_mb):
    # Ensure the directory ends with a slash
    if not directory_path.endswith('/'):
        directory_path += '/'

    # Generate exact-sized Excel and PDF files
    create_exact_size_file(directory_path + excel_filename, target_size_mb)
    create_exact_size_file(directory_path + pdf_filename, target_size_mb)

    print(f'Generated Excel file at {directory_path + excel_filename} with size exactly {target_size_mb} MB')
    print(f'Generated PDF file at {directory_path + pdf_filename} with size exactly {target_size_mb} MB')

# Set the directory where files will be saved
directory_path = "/Users/ikawahyuni/dataquality/dummy_files"

# Set filenames and target size
generate_files(directory_path, 'dummy_15mb.xlsx', 'dummy_15mb.pdf', 15)


# 14 MB

In [1]:
import os

def create_exact_size_file(file_path, size_mb):
    # Calculate the exact size in bytes
    size_bytes = size_mb * 1024 * 1024

    # Open the file in write binary ('wb') mode
    with open(file_path, 'wb') as file:
        file.seek(size_bytes - 1)  # Move to the position one byte before the desired size
        file.write(b'\0')  # Write a single zero byte at this position

def generate_files(directory_path, excel_filename, pdf_filename, target_size_mb):
    # Ensure the directory ends with a slash
    if not directory_path.endswith('/'):
        directory_path += '/'

    # Generate exact-sized Excel and PDF files
    create_exact_size_file(directory_path + excel_filename, target_size_mb)
    create_exact_size_file(directory_path + pdf_filename, target_size_mb)

    print(f'Generated Excel file at {directory_path + excel_filename} with size exactly {target_size_mb} MB')
    print(f'Generated PDF file at {directory_path + pdf_filename} with size exactly {target_size_mb} MB')

# Set the directory where files will be saved
directory_path = "/Users/ikawahyuni/dataquality/dummy_files"

# Set filenames and target size (change from 15 to 14)
generate_files(directory_path, 'dummy_14mb.xlsx', 'dummy_14mb.pdf', 14)


Generated Excel file at /Users/ikawahyuni/dataquality/dummy_files/dummy_14mb.xlsx with size exactly 14 MB
Generated PDF file at /Users/ikawahyuni/dataquality/dummy_files/dummy_14mb.pdf with size exactly 14 MB


# maximum size limit of 10,485,760 bytes (10 MB)

In [3]:
import os

def create_exact_size_file(file_path, size_mb):
    # Maximum size in bytes (10 MB)
    max_size_bytes = 9485760

    # Calculate the desired size in bytes
    size_bytes = size_mb * 1024 * 1024

    # Adjust size if it exceeds the maximum allowed
    if size_bytes > max_size_bytes:
        size_bytes = max_size_bytes

    # Open the file in write binary ('wb') mode
    with open(file_path, 'wb') as file:
        file.seek(size_bytes - 1)  # Move to the position one byte before the desired size
        file.write(b'\0')  # Write a single zero byte at this position

def generate_files(directory_path, excel_filename, pdf_filename, target_size_mb):
    # Ensure the directory ends with a slash
    if not directory_path.endswith('/'):
        directory_path += '/'

    # Generate exact-sized Excel and PDF files
    create_exact_size_file(directory_path + excel_filename, target_size_mb)
    create_exact_size_file(directory_path + pdf_filename, target_size_mb)

    actual_size_mb = min(target_size_mb, 10)  # Limit the reported size to 10 MB max
    print(f'Generated Excel file at {directory_path + excel_filename} with size up to {actual_size_mb} MB')
    print(f'Generated PDF file at {directory_path + pdf_filename} with size up to {actual_size_mb} MB')

# Set the directory where files will be saved
directory_path = "/Users/ikawahyuni/dataquality/dummy_files"

# Set filenames and target size (e.g., 12 MB would be reduced to 10 MB)
generate_files(directory_path, 'dummy.xlsx', 'dummy.pdf', 12)


Generated Excel file at /Users/ikawahyuni/dataquality/dummy_files/dummy.xlsx with size up to 10 MB
Generated PDF file at /Users/ikawahyuni/dataquality/dummy_files/dummy.pdf with size up to 10 MB


## Tool to collect all tickets 

In [12]:
import os
import zipfile
import rarfile  # pip install rarfile
import pandas as pd

def extract_and_process_files(directory):
    # Iterate over all the files in the given directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        extract_path = os.path.join(directory, filename[:-4])  # Folder to extract files

        # Check file type and extract accordingly
        if filename.endswith(".zip"):
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
                print(f"Extracted {filename} to {extract_path}")
        elif filename.endswith(".rar"):
            with rarfile.RarFile(filepath, 'r') as rar_ref:
                rar_ref.extractall(extract_path)
                print(f"Extracted {filename} to {extract_path}")
        else:
            continue
        
        # Look for the specific ReportTest.arx file in the extracted files
        report_file_path = find_report_file(extract_path)
        if report_file_path:
            df = create_dataframe_from_file(report_file_path, extract_path)
            csv_path = os.path.join(directory, filename[:-4] + '_files_info.csv')
            df.to_csv(csv_path, index=False)
            print(f"Data from {report_file_path} saved to CSV at {csv_path}")
        else:
            print(f"No 'ReportTest.arx' found in {extract_path}")

def find_report_file(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == 'ReportTest.arx':
                return os.path.join(root, file)
    return None

def create_dataframe_from_file(file_path, extract_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    titles = []
    descriptions = []
    version_data = []
    first_publish_location_ids = []

    for line in lines:
        if line.startswith("DATA"):
            parts = line.strip().split('"')
            ticket_number = parts[1].strip()
            file_name = parts[3].strip().partition(' ')[2]  # Extracting everything after "1 "

            full_path = os.path.join(extract_path, file_name)

            titles.append(file_name)
            descriptions.append("")
            version_data.append(full_path)
            first_publish_location_ids.append(ticket_number)

    return pd.DataFrame({
        'Title': titles,
        'Description': descriptions,
        'VersionData': version_data,
        'FirstPublishLocationId': first_publish_location_ids
    })

# Path to your directory containing ZIP and RAR files
directory_path = 'D:\Salesforce\dataquality\Tool for attachments' 
extract_and_process_files(directory_path)


Extracted ReportTest.zip to D:\Salesforce\dataquality\Tool for attachments\ReportTest
Data from D:\Salesforce\dataquality\Tool for attachments\ReportTest\ReportTest.arx saved to CSV at D:\Salesforce\dataquality\Tool for attachments\ReportTest_files_info.csv
